サーバーもラップトップもまた電話ですら、今日ではプログラムが使うことができる制御の独立した複数のスレッドによるマルチコアのチップで作られている。
プログラムがそれらのチップのすべての長所を引き出すように設計する必要がある。
Clojureはこのマルチコアの世界のなかでそのために生まれた。

Javaのような言語における並列性の大部分の問題は共有する可変的な状態を管理する問題にある。
これまで見てきたように、Clojureは基本的に(複数のスレッド間で安全に使うことができる)不変的なデータに頼っている。
また、これも見てきたように、状態をもつコンテナ(`atom`、`ref`、`agent`、`var`)を使い、明白な状態を作ることができる。
すべてのコンテナは一般的な更新モデルを使い、それにより状態は純粋な関数によってひとつの不変的な値から別の値へいつでも変換される。
この多くの一般的なエラーのクラスを取り除くアプローチの組み合わせにより、それらすべてコアが何かしら働かせる方法の本当の問題に焦点を合せることが可能になる。

わたしたちが出会う最初の問題のひとつは、主なスレッドから作業を移すことと主なスレッドがその仕事をする間に非同期に実行する方法です。
一度あれをすれば、その非同期なタスクが仕事を終えたときにその結果を受け取る方法も必要とする。
Clojureの`future`と`promise`へと飛び込みます。

長寿命なタスク指向な並列性のために、一連のタスクをワーカー・スレッドのプールに振り分けて処理します。
JavaはClojureから直接呼び出せるキューとワーカーのための頑強な道具を持っている。
その道具によって全てのコアを使って仕事の流れを効果的に処理することが可能になる。

いくつかの場合において、並列に、複数のコレクションに渡り、同じ変換をすべての要素に行う、きめ細かい仕事を実装する必要がある。
それらの問題をコレクションとシーケンス関数によってアプローチする方法をすでに見てきたが、Clojureは`reducer`と呼ばれる他のオプションを持っている。
`reducer`によってまるでシーケンスであるかのように変換を構成するがその実行は並列であることが可能になる。

最後に、プログラムの全体的な構造を組織するためにどのようにスレッド(と軽量プロセス)を使うことができるか考えたい。
`core.async`ライブラリはその構成を助けるために`channel`と`go block`の概念を定義する。
アプリケーションの構造を定義する方法を見る。

# Push Waiting to the Background

大部分のプログラムは外の世界とファイルやソケットや標準終端ストリームを通してつながる。
それらの入力と出力をI/Oと呼ぶ。
現代のプロセッサは一秒に何十億もの命令を実行できるが、大部分のI/Oは比較的低速です。
たくさんのプログラムがファイルからデータを読みだしや外部サーバからの応答の受信やユーザがやりたいことを見付けるののを待つためのかなりの時間を過す。

この街を効果的に行うことが必要であり、プログラムは他の仕事を続けられる。
待っているあいだ、他の処理を実行することもできるし、同じことを並列で待つこともできる。
例えば、ウェブ・ブラウザは外部のウェブ・サーバがコンテンツを返すのを待つために時間を過すひとつのプログラムです。一方で同時に、現在のページを表示し、リンクのクリックへの応答をし、ページのスクロールをしたりする。

## Fire and Forget

はじめに、応答の必要がないバックグラウンドでなされる必要がある簡単な仕事の場合を考えよう。
アプリケーションを組み立てて、イベントが発生するたびに外部のメーター法のコレクタを呼びたいと想像してください。
外部のサービスを`inc-stat`という便利な関数に包むことができる。
状態を更新するためにそれを呼び出す。

```
(inc-stat :pageview)
```

この関数はネットワークごしに外部のウェブ・サーバを呼ぶことになる。
ページ・ビューを生成しているあいだにそれを呼んだとき、呼び出しをする時間は各ページを構築するのが遅くなる。下の図から分るように。

![fig 05-01](fig_05_01.jpg)

この仕事をバックグラウンドのスレッドに移動するには、Clojureに含まれる`future`関数を使う。

```
(future (inc-stat :pageview))
```

`future`関数はボディを1つ取り、Clojureが管理するバックグラウンド・スレッド・プールにそのボディを呼び出す。
その違いを図で見ることができる。

![fig 05-02](fig_05_02.jpg)

また、ボディを渡す代りに非同期に引数なしの関数を呼び出す`future-call`を使うこともできる。
どちらの関数も非同期な活動を制御し検査する`java.lang.Future`オブジェクトを返す。
`future-cancel`関数はその実行をキャンセルする、一方`future-done?`と`future-cancelled?`はその状態の情報を与える。

しかしながら、遠隔地のサービスへ独立した統計的増加メッセージの洪水を送ることは非効率的に見える。
送信する前にいくつかの増分メッセージをまとめておくほうが理にかなっています。(??)
そうするためには、非同期でかつ状態をもつことが必要です。

## Asynchronous and Stateful

「State, Identity and Change」において、Clojureにおけるいくつかの状態コンテナ(`var`、`atom`、`ref`)を調べた。
われわれはもうひとつの状態コンテナである`agent`の導入をここまで引き伸ばした。

他の状態コンテナのように、`agent`は不変的な値を保持し、同じ更新モデルを使って修正される。
他のコンテナと違うのは、`agent`は非同期に更新されるということです。

メーター法コレクタを考えてください。
特定の統計のためのカウンタを`agent`に保持しましょう。

In [1]:
(def pageview-stat (agent 0))

#'user/pageview-stat

すべての`agent`の更新ごとに遠隔サーバを呼ぶのでなく、10回ごとにだけ呼ぶとします(`agent`の状態が10で割り切れる数に逹したとき)。
これは監視をしやすい(すべての状態コンテナで働く)。

In [3]:
(defn remote-send [key new-val]
  ,,, )
(add-watch
    pageview-stat
    :pageview
    (fn [key agent old new]
        (when (zero? (mod new 10))
            (remote-send key new))))

#agent[{:status :ready, :val 0} 0x515d3428]

ここで、状態へのなにかしらの変更により発火する`pageview-stat``agent`へ監視を加えた。
今、新しい`agent`の状態が10の倍数のときだけ外部サービスの要求の発火、それはわれわれにいくつかのバッチ処理を与える。(???)

そして、ちょうどその`agent`上で非同期に実行される関数が送るのようにアプリケーションが使う増加関数を定義できる。(??)


In [1]:
(defn inc-stat [stat]
    (send-off stat inc))

#'user/inc-stat

Clojureは`agent`上にて非同期に更新処理を呼び出す2つの関数を提供する。`send`と`send-off`です。
計算量が多く、入出力をブロックしない`agent`の更新には`send`を使用してください。(??)
基礎になるスレッド・プールはスレッドの固定された組を使い、ちょうど間に合った方法の中でこれらの更新が完了することに依存する。
任意の時間、ブロックする更新のためには`send-off`を使用してください。
基礎になるスレッド・プール(将来のためにも使う)は必要に応じて増えるので、ブロッキングは問題になりません。
`inc-stat`において、外部のサービスを(`agent`のスレッドにて実行する監視を経由し)潜在的に呼び出しているので、`send-off`を使います。

`agent`のひとつの追加的な特徴は、STMトランザクションの内部`agent`上または`agent`の活動自体の内部で呼び出される`send`や`send-off`はトランザクションが完了するまで遅らされる。
こうして、副作用を生産するために、(成功するために再試行が必要かもしれない)STMトランザクションの内側またはその他の`agent`の更新活動の内側で`agent`を安全に呼び出すことができる。

    #### Shutting Down
    
    Javaにおいて、スレッドはダエモン・スレッドとしてマークされる。
    JVMはすべてのダエモンでない・スレッドが仕事を終えたとき(典型的にこれは主要な開始するスレッドが終了したときに発生する)優雅にシャット・ダウンする
    スレッドをダエモンスレッドとしてマークすることはそれはバックグラウンドに仕事し、シャットダウンを妨げるべきでないことを意味する。

    `future`と`agent`の活動を処理するスレッドはダエモン・スレッドではない。
    もしアプリケーションが想定するとおりに終了する代りにハングするようならば、アプリケーションが終了するまでの間、`shutdown-agent`の呼び出しを追加することが必要だろう。

これまで、なにも応答せずに、バックグラウンドにブロックする活動をしてきた。
バックグラウンド・スレッドで行われた仕事から、どのように応答を受け取ることができるのか見ましょう。

## Waiting for a Response

ときどき、仕事をバックグラウンドに移して後でその結果を取りに戻りたいと思う。
例えば、オンライン・ストアにて、いくつかの製品を取り、それらの価格を比べたいというアプリケーションを考えてください。
それぞれの店に順番にシングルスレッドで問い合せると、次のようになる。


In [6]:
(defn query [store product] ,,,)

(defn query-stores [product stores]
    (for [store stores]
        (query store product)))

#'user/query-stores

これを実行する時間は各店に問い合わせる時間の合計であり、図示すると次のようになる。

![fig-05-03](fig_05_03.jpg)

もっと上手くできます!
仕事をバックグラウンド・スレッドに移す魔法のような可能性により、すべての店へ同時に問い合わせることができる。
非同期な問い合わせのために`future`を使う。


In [5]:
(defn query-stores [product stores]
    (for [store stores]
        (future (query store product))))

#'user/query-stores

しかしながら、すでに述べたとおり、`futre`は`java.lang.Future`を返すので、`query-store`は今、それらのシーケンスを返す。実際の結果ではない。
非同期な呼び出しの結果をブロックして待つために、`future`を`deref`または短縮形の`@`により間接参照する必要がある。

ここで`query-store`関数を2段階で実装した。
(すでに)第一段階で、すべての問い合わせに着手し、`future`オブジェクトのシーケンスを生産した。
必要とする唯一の変更は、`doall`を呼ぶことでその実行を(lazyでなく)強制することです。
もし`doall`を呼ばないならば、その問い合わせは後でシーケンスが実体化されるまで着手されない。
第二段階で各`future`を間接参照し、それぞれが完了するまでブロックする。

In [7]:
(defn query-stores [product stores]
    (let [futures (doall
                      (for [store stores]
                          (future (query store product))))]
        (map deref futures)))

#'user/query-stores

この`query-stores`関数は各店からの結果のlazyシーケンスを返す。
また、それらを間接参照しない`futures`のlazyシーケンスを返すだろう。(??)
そして、呼び元は各結果を解決するためにいつブロックするかを完全に制御できるだろう。(??)

今、問い合わせを並列に実行している。同時に、いくつかのサービスを待つことができる。もっとスレッドの影響力を行使することで全体の時間を削減する。次の図のように。
この図は`future`に呼び出され、各問い合わせの結果を間接参照することを待っている3つの`query-stores`を示している。

![fig-05-04](fig_05_04.jpg)

この方法で`future`を使うことはひとつのマイナス面をもつ。非同期な計算は元のコードへひとつの結果を返すだけです。
もし計算上の異なる点での複数の値を返して欲しいならば、`promise`を使うことができる。

## Making Promises

`promise`はひとつの値(だけ)をあるスレッドから別のスレッドへ移すために使われます。
従って、複数の`promise`は非同期な計算における異なる時点における値を返すために使われます。
例えば、ある非同期な計算に時間をかけて、その後その実行の開始と終了の時間を取得したいと思ってください。

In [1]:
(defn long-running-task []
    ,,,)

(defn launch-timed []
    (let [begin-promise (promise)
          end-promise (promise)]
        (future (deliver begin-promise (System/currentTimeMillis))
            (long-running-task)
            (deliver end-promise (System/currentTimeMillis)))
        (println "task begin at" @begin-promise)
        (println "task end at" @end-promise)))

#'user/launch-timed

この例において、`begin-promise`と`end-promise`の2つの`promise`を作る。
値は`promise`を経由して`deliver`関数により届けられる。
値はいちどだけ`promise`を経由して届けられ、最初の配達の後、その次の配達はなんの影響もない。

それらの`promise`は後でその値を取り戻すために間接参照できる。
この間接参照は値が利用可能である間ブロックする。そしてそれを返す。
`promise`の値が利用可能であるかブロッキングなしで見るためには、`realize?`関数を使う。
また、`deref`の変種が特定のタイムアウト期間を待つことに気を付けてください。

`future`と`agent`を仕事を非同期におこなうために使えて、待つことを押しやるかバックグラウンド・スレッド上にて仕事をすることが可能になる。
`future`と`promise`は同様に、非同期タスクから結果を返す方法を制御することを可能にしてくれる。
(それがあなたのためになるかもしれないものは何でも)実際の仕事をおこなうプログラムを構築する方法を見ることが必要である。

# Queues and Workers

多くのプログラムはタスク・プロセッサの全体または一部に見える。タスクとは普通外部の要求に対応する仕事の単位である。
ウェブ・アプリはウェブ・ページを構築する要求を受ける。
ウェブ・サーバはAPI呼び出しの処理をする要求を受ける。
バッチ・プログラムはディスクまたはデータベースからファイルを読み出して、それらを適切に処理する。
これら一般的なパターンのすべては、ワーカーのプールに養殖された仕事のキューとしてモデル化される。

キューはタスクを順序付けして保持し、仕事が到着した場所と処理された場所を分離します。
ワーカー・プールによって異る属性のワーカーのプールを作り、その仕事を管理し監視することに使用される並列性の量とポリシーを制御することが可能になります。(??)
あの制御により廃棄におけるハードウェアの使用をフルにすることが可能になる。(???その管理により、私たちは自分の処分でワードウェアを最大限に活用することができます。)

Clojureはキューとワーカーに対していくつかの道具を提供するが、また、Javaですでに利用可能な高品質な道具の再発明を避ける。
Clojureでこれまでに見てきた部品からキューとワーカー・プールを作る方法を考えよう。

## Some Assembly Required

「Model Your Domain」にて、FIFOデータのためにリストやベクタよりさらに効果的なアクセスを提供するためにClojureの永続的なキューを使った。
しかしながら、シングル・スレッドの文脈のなかでそうした。
永続的なキューにおいて、キューの各変形は更新された版を返す。
もし複数のスレッドがキューを共有すれば、それらはみな同じインスタンスを共有することを必要とする。
(`atom`や`ref`のような)状態管理の構造と状態をもったキューの実装の両方が必要である。

われわれの選択肢はClojureの`atom`か参照に永続的なキューを包むことであろう。それはキューの両端が安定したアイデンティティを維持する。
もしこれを`atom`で試みるならば、`atom`の`swap!`関数は`atom`(わたしたちにとってのキュー)の新しい値を返せるだけで、弾かれた値ではないと気づくだろう。(??)
これによりこの主のキューから状態を持った方法で項目を引き出すことが困難になる。(??)

`ref`の選択はより有望にみえる。
このように実装できた。

In [2]:
(defn queue
    "Create a new stateful queue"
    []
    (ref clojure.lang.PersistentQueue/EMPTY))

(defn enq
    "Enqueue item in q"
    [q item]
    (dosync
        (alter q conj item)))

(defn deq
    "Dequeue item from q (nil if none)"
    [q]
    (dosync
        (let [item (peek @q)]
            (alter q pop)
            item)))

#'user/deq

しかしながら、このキューはブロックしない。


## Java Queues

# Parallelism with Reducers

## From Sequences to Reducers
## Reducer Performance

# Thinking in Processes

## Channels
## Go Blocks
## Pipelines

# Wrapping Up
